<h4>Import liberies</h4>

In [66]:
import os
import numpy as np
import pandas as pd
import scipy.cluster.hierarchy as sch
import seaborn as sns
import matplotlib.pyplot as plt
import itertools 

<h5>house keeping params</h5>

In [16]:
RANDOM_STATE = 24
np.random.seed(RANDOM_STATE) # Set Random Seed for reproducible  results

NOISE = 0.1
N_SAMPLES = 1000
ALPHA = 0.001

# parameters for Matplotlib
params = {'legend.fontsize': 'small',
          'figure.figsize': (15, 7),
          'axes.labelsize': 'large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'large',
          'ytick.labelsize':'large',
          'savefig.dpi': 75,
          'image.interpolation': 'none',
          'savefig.bbox' : 'tight',
          'lines.linewidth' : 1,
          'legend.numpoints' : 1
         }

CMAP = plt.cm.rainbow
plt.rcParams.update(params);
plt.set_cmap(CMAP);

plt.style.use('seaborn-v0_8-darkgrid') # plt.style.use('ggplot')

TEST_SIZE = 0.25

<Figure size 1500x700 with 0 Axes>

<h4>Get Data</h4>

In [22]:
groceries_df = "groceries_mba.csv"

In [28]:
with open(groceries_df) as file:
    lines = file.readlines()
    lines = [sorted(list(set(line.rstrip().split(',')))) for line in lines]

lines[:10]

[['citrus_fruit', 'margarine', 'ready_soups', 'semi-finished_bread'],
 ['coffee', 'tropical_fruit', 'yogurt'],
 ['whole_milk'],
 ['cream_cheese_', 'meat_spreads', 'pip_fruit', 'yogurt'],
 ['condensed_milk',
  'long_life_bakery_product',
  'other_vegetables',
  'whole_milk'],
 ['abrasive_cleaner', 'butter', 'rice', 'whole_milk', 'yogurt'],
 ['rolls/buns'],
 ['UHT-milk',
  'bottled_beer',
  'liquor_(appetizer)',
  'other_vegetables',
  'rolls/buns'],
 ['pot_plants'],
 ['cereals', 'whole_milk']]

In [57]:
supps = []

supp = {}

for line in lines:
    for key in line:
        if key in supp:
            supp[key] += 1
        else:
            supp[key] = 1

In [60]:
SUPP_THRESHOLD = 100
CONF_THRESHOLD = 0.4
LIFT_THRESHOLD = 20.0
CONV_THRESHOLD = 5.0

In [62]:
f_supp = {k: v for k,v in supp.items() if  v >= SUPP_THRESHOLD}
len(f_supp)

88

In [64]:
supps.append(f_supp)

In [95]:
    supp = {}

    for line in  lines:

        for combination in itertools.combinations(line, 2):

            if combination[0] in supps[0] and combination[1] in supps[0]:
                key = ','.join(combination)
                if key in supp:
                    supp[key] += 1
                else:
                    supp[key] = 1

f_supp = {k: v for k,v in supp.items() if  v >= SUPP_THRESHOLD}
supps.append(f_supp)

In [106]:
    supp = {}

    for line in  lines:

        for combination in itertools.combinations(line, 3):

            if combination[0] in supp[0] and combination[1] in supps[0]:
                key = ','.join(combination)
                if key in supp:
                    supp[key] += 1
                else:
                    supp[key] = 1

f_supp = {k: v for k,v in supp.items() if  v >= SUPP_THRESHOLD}
supps.append(f_supp)

KeyError: 0

In [108]:
def measures(supp_ab, supp_a, supp_b, transaction_count):
    conf = np.float32(supp_ab)/ supp_a
    sup = np.float32(supp_b)/ transaction_count
    lift = conf/ sup
    if conf == 1.0:
        conv = np.float32('inf')
    else:
        conv = (1-sup)/(1-conf)
    return [conf, sup, lift, conv]

In [110]:
transaction_count = len(lines)
rules = []
for i in range(2, len(supps)+1):
    for key, value in supps[i-1].items():
        key + key.split(',')
        for j in range(1, len(key)):
            for a in intertools.combinations(key,j):
                b= tuple([w for w in key if w not in a])
                [conf, sup, lift, conv]= measures(value,supps[len(a)-1][','.join(a)],
                                                        supps[len(b)-1][','.join(b)],
                                                        transaction_count)
                if conf>= CONF_THRESHOLD:
                    rules.append((a,b,conf,lift,conv))
        rules = sorted(rules, key = lambda x: [x[0],x[1]])
        rules = sorted(rules, key = lambda x: (x[2]), reverse=True)

TypeError: can only concatenate str (not "list") to str